<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
f = ZipFile("/content/drive/MyDrive/news/chinese_news_trans.zip")
f.extractall()
f = ZipFile("/content/drive/MyDrive/news/chinese_news_test.zip")
f.extractall()

In [ ]:
import os
import glob
import pandas as pd
def process(base):
    datas = {
        "content":[],
        "ans":[]
    }
    dirs = glob.glob(os.path.join(base, "*"))
    for d in dirs:
        fns1 = os.path.join(d, "*.txt")
        fns2 = os.path.join(d, "*.TXT")
        fns = glob.glob(fns1) + glob.glob(fns2)
        for fn in fns:
            with open(fn, "r", encoding="utf-8") as f:
                datas["content"].append(f.read())
            datas["ans"].append(os.path.split(d)[-1])
    df = pd.DataFrame(datas)
    return df
train_df = process("chinese_news_trans")
test_df = process("chinese_news_test")

In [ ]:
train_df

In [ ]:
# Series.replace({"原本":"改成"})
# Series.unique/value_counts
u = train_df["ans"].unique()
name2cat = {name:i for i, name in enumerate(u)}
cat2name = {i:name for i, name in enumerate(u)}
y_train = train_df["ans"].replace(name2cat)
y_test = test_df["ans"].replace(name2cat)
y_test

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")

In [ ]:
# Series.apply(func)
import jieba
jieba.set_dictionary("dict.txt.big")
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
pre = clf.predict(x_test_count)
print(accuracy_score(y_test, pre))
prob = clf.predict_proba(x_test_count)
print(log_loss(y_test, prob))

In [ ]:
u

In [ ]:
p = input("write:")
test = vec.transform([newscut(p)])
prob = clf.predict_proba(test)[0]
for i in range(len(u)):
    print(u[i], "的機率:", round(prob[i], 3))